# 2 - Aprendizaje por refuerzo

**Sumario**

1. Introducción
2. El framewok de aprendizaje por refuerzo
3. Métodos de aprendizaje por refuerzo basados en valor
4. Q-Learning
5. Deep Reinforcement Learning
6. Construyendo un agente mediante una red DQNN

## 2.1 - Introducción

Cuando reflexionamos sobre la palabra **aprendizaje**, es posible que lo primero que nos venga a la mente sea **la forma en la que los seres humanos aprendemos mediante la interacción con nuestro entorno**.

Así, uno de los primeros ejemplos que seguramente visualicemos sea el proceso de aprendizaje que experimentan los niños y las niñas cuando empiezan a andar, donde interactúan con el entorno mediante un modelo de causa y efecto que voluciona con experimentación. De este modo, los seres humanos vamos adaptando poco a poco nuestro sistema motriz cada vez que se producen caidas (**"penalizan" el modelo**) o que conseguimos alcanzar una determinada localización en nuestro entorno (**"refuerzan"** el modelo).

<table>
    <tr>
        <td><img src="images_2/humans.jpg" width="500" data-align="center"></td>
    </tr>
</table>

Dicho sistema de **penalización** y **refuerzo** se utilizó como inspiración para definir **el aprendizaje por refuerzo** (Sutton, 1998). El cual define un modelo de comportamiento que simula el proceso de interacción entre un agente y un entorno mediante la utilización de estímulos (refuerzos) que le indicasen cuales son las decisiones mas prometedoras para alcanzar un determinado objetivo.

Comenzó con acermientos basados en **programación dinámica**, y luego fue evolucionando hacia los **algoritmos basados en tablas**, como *Q-Learning* o *SARSA*, los cuales construyen una tabla (normalmente llamada **tabla Q**) donde para cada estado, se incluía un valor de calidad para cada una de las posibles acciones que se podrian realizar en un determinado estado.

Este tipo de acercamientos implicaban un gran **problema computacional** ya que hacia que muchos problemas resultasen intratables por la **combinación de estados y acciones que definía el tamaño de la tabla**. Este inconveniente se solventó con el surgimiento del aprendizaje profundo (*Deep Learning*), permitiendo que el aprendizaje por refuerzo pudiera resolver problemas antes inabordables, es decir, problemas donde el tamaño de la tabla Q seria demasiado grande.

En el **aprendizaje por refuerzo profundo** (***Deep Reinforcement Learning***) utilizamos una red neuronal en vez de una tabla Q para dar un valor "aproximado" de la calidad de una acción para un determinado estado. Este tipo de acercamiento fue un gran avance en el área, permitiendo desarrollar modelos para tareas impensables hasta el momento, como videojuegos (e.g., [**Atari**](https://openai.com/research/gym-retro) o [**Starcraft II**](https://www.deepmind.com/blog/alphastar-grandmaster-level-in-starcraft-ii-using-multi-agent-reinforcement-learning)), [**el juego de Go**](https://www.youtube.com/watch?v=WXuK6gekU1Y) o [**el plegamiento de proteinas**](https://www.deepmind.com/research/highlighted-research/alphafold).

<table>
    <tr>
        <td><img src="images_2/alphago.jpg" width="500" data-align="center"></td>
    </tr>
</table>

A lo largo de esta unidad, describiremos los conceptos básicos asociados al aprendizaje por refuerzo y estudiaremos cómo este ha evolucionado desde los algoritmos tradicionales hasta las redes de neuronas por refuerzo, que utilizaremos, finalmente, para construir un agente que aprenda a jugar a un videojuego.

## 2.2 - El framework de aprendizaje por refuerzo

La idea detrás del aprendizaje por refuerzo es que un **agente** (una IA) aprenderá del **entorno** interactuando con él (a través de prueba y error) y recibiendo **recompensas** (negativas o positivas) como retroalimentación por realizar acciones.

Por ejemplo, imagina poner a tu hermano pequeño frente a un videojuego que nunca jugó, darle un controlador y dejarlo solo. Tu hermano podria interactuar con el "entorno" (el videojuego) pulsando los botones del mando. Si obtiene una moneda, recibirá un recompensa positivo (e.g., +1). En cambio, si toca un enemigo, recibirá una recompensa negativa (e.g., -1).

Al interactuar con su entorno a través de prueba y error, tu hermano acabaría entendiendo que necesita recolectar monedas y evitar enemigos. Sin ninguna supervisión el niño mejoraría  cada vez más en el juego.

<table>
    <tr>
        <td><img src="images_2/RL_process_game.jpg" width="500" data-align="center"></td>
    </tr>
</table>

* **Agente**. La entidad que interactúa con el entorno obteniendo información de éste y modificándolo  mediante acciones.
* **Entorno**. La representación virtual del mundo con el que interactúa el agente.
* **Estado**. La representación completa del entorno en un instante específico de tiempo, así como la representación del agente en dicho instante.
* **Acción**. Aquella que ejecuta el agente ene el entorno con el fin de producir una variación en este.
* **Recompensa o refuerzo**. Valor numérico obtenido tras la ejecución de una acción en el entorno. Se utiliza como medida para evaluar si la ejecución de la acción ha resultado positiva o negativa para el agente

**Este proceso es un bucle, donde el objetivo del agente es maximizar la recompensa acumulada, que representa el retorno esperado**.

Hay varios componentes/asunciones relevantes en el framework del aprendizaje por refuerzo (los veremos ahora uno por uno):
* La hipótesis de recompensa.
* La propiedad de Markov.
* El grado de observabilidad del entorno (total o parcial).
* El espacio de acción (finito o infinito).
* Las recompensa y el factor de descuento.
* *Tradeoff* entre exploración y explotación
* Enfoque de aprendizaje (basado en política o en valor)

### 2.2.1 - La hipótesis de recompensa

El aprendizaje por refuerzo se basa en la **hipótesis de recompensa**, la cual indica que **cualquier meta puede describirse como la maximización del rendimiento esperado** (recompensa acumulada esperada).

Es por eso que en el aprendizaje por refuerzo, para tener **el mejor comportamiento**, nuestro objetivo es aprender a **tomar acciones que maximicen la recompensa acumulada esperada**.

### 2.2.2 - La propiedad de Markov

El proceso de RL también se denomina **Proceso de decisión de Markov** (*Markov Decission Process*, *MDP* por sus siglas en inglés).

La propiedad de Markov implica que **nuestro agente solo necesita el estado actual para decidir qué acción tomar** y no el historial de todos los estados y acciones que tomó antes.

### 2.2.3 - El grado de observabilidad del entorno

<table>
    <tr>
        <td><img src="images_2/obs_space.jpg" width="600" data-align="center"></td>
    </tr>
</table>

-----

**Nota**: Ciertos autores hacen la distinción entre "estado" y "observación" para nombrar a $S_{t}$ en el bucle de aprendizaje por refuerzo.

-----

### 2.2.4 - El espacio de acción

<table>
    <tr>
        <td><img src="images_2/action_space.jpg" width="600" data-align="center"></td>
    </tr>
</table>

### 2.2.5 - La recompensa y el factor de descuento

La recompensa es un elemento fundamental en el proceso de aprendizaje automático porque **es el único feedback que recibe nuestro agente**. Gracias a ella, sabe si esta acción ha sido **positiva o negativa**.

La recompensa acumulada para un instante de tiempo $t$ se puede escribir como:

<table>
    <tr>
        <td><img src="images_2/rewards_1.png" width="400" data-align="center"></td>
    </tr>
</table>

Esto es equivalente a:

$$
R(\tau) = \sum_{k=0}^{\infty} r_{t+k+1}
$$

Sin embargo, en realidad, no podemos simplemente agregar las recompensas así. Las recompensas que llegan antes (al comienzo del juego) **tienen más probabilidades de suceder**, ya que son más predecibles que las recompensas futuras a largo plazo.

Digamos que tu agente es este pequeño ratón que puede mover una ficha en cada paso de tiempo, y tu oponente es el gato (que también puede moverse). **El objetivo del ratón es comer la máxima cantidad de queso antes de ser comido por el gato.**

En consecuencia, la recompensa cerca del gato, aunque sea más grande (más queso), **estará más rebajada ya que no estamos muy seguros de poder comérnoslo**.

<table>
    <tr>
        <td><img src="images_2/rewards_3.jpg" width="400" data-align="center"></td>
    </tr>
</table>

Para descontar las recompensas, procedemos así:

1. Definimos una tasa de descuento $\gamma$. Debe estar entre 0 y 1. Normalmente suele tomar valores cercanos a 0.99 o 0.95.
    * Cuanto mayor sea $\gamma$, **menor será el descuento**. Esto implica que el agente se centra más la **recompensa futura**.
    * cuanto menor sea $\gamma$, **mayor será el duescuento**. Esto implica que el agente se centra más en la **recompensa cercana**.
2. Luego, cada recompensa será descontada por $\gamma$ elevada al instante de tiempo actual. A medida que aumenta el paso de tiempo, el gato se acerca a nosotros, por lo que la recompensa futura es cada vez menos probable.

La recompensa acumulada (**descontada**) para un instante de tiempo $t$ se puede escribir como:

<table>
    <tr>
        <td><img src="images_2/rewards_2.png" width="400" data-align="center"></td>
    </tr>
</table>

$$
R(\tau) = \sum_{k=0}^{\infty} \gamma^{k} r_{t+k+1}
$$

### 2.2.6 - *Tradeoff* entre exploración y explotación

Antes de describir los diferentes enfoques de aprendizaje, debemos entender un aspecto muy importante en los problemas de aprendizaje por refuerzo: **el *tradeoff* entre exploración y explotación**.

Recordemos que el objetivo de nuestro agente es el de maximizar la recompensa acumulada. Sin embargo, **podemos caer en una trampa común**. Para entenderla, consideremos el siguiente juego donde nuestro ratón puede tener una cantidad *infinita* de queso (+1 por cada uno que recoge), pero en la parte superior del "laberinto" hay una cantidad gigante de queso (+1000).

<table>
    <tr>
        <td><img src="images_2/exp_1.jpg" width="400" data-align="center"></td>
    </tr>
</table>

Si solo nos centramos en **recoger el queso más cercano sin explorar nuestro entorno**, nuestro agente nunca llegara a la suma gigante de queso. Sin embargo, **si realiza algo de exploración  (perdiendo algo de recompensa en el corto espacio de tiempo) podriamos conseguir una recompensa aún mayor**. Esto es lo que denominamos como el *tradeoff* entre exploración y explotación.

<table>
    <tr>
        <td><img src="images_2/expexpltradeoff.jpg" width="600" data-align="center"></td>
    </tr>
</table>

### 2.2.7 - Enfoque de aprendizaje

**La política $\pi$ es la que define el comportamiento del agente de tal forma que sus acciones maximicen la recompensa acumulada**. Podemos verla como el **"cerebro" del agente**.

Nuestro objetivo es encontrar la política óptima $\pi^{*}$, la cual **maximiza la el retorno esperado** (i.e., la recompensa acumulada) cuando el agente actúa de acuerdo con ella. **La política óptima $\pi^{*}$ se encuentra mediante el entrenamiento.**

Hay dos posibles acercamientos para entrenar a nuestro agente a encontrar la política óptima $\pi^{*}$:
* **Directamente**, enseñando al agente que acción debe tomar en cada estado: **Policy-based methods**.
* **Indirectamente**, enseñando al agente que estado es más valioso, de tal forma que tome acciones que le lleven a estados valiosos: **Value-based methods**.

<table>
    <tr>
        <td><img src="images_2/two_approaches.png" width="600" data-align="center"></td>
    </tr>
</table>

## 2.3 - Métodos de aprendizaje por refuerzo basados en valor

Aprendemos una función que asigna a cada estado el valor esperado de estar en él. Dicho valor es el rendimiento descontado esperado que el agente puede obtener si comienza en ese estado y luego actúa de acuerdo con nuestra política.

-----

<span style="color:red"><b>Pregunta:</b></span> Pero, ¿que significa actuar con respecto a nuestra política? Al fin y al cabo, no tenemos una política en métodos basados en valor dado que lo que aprendemos es una función de valor, no una política en sí misma.

<span style="color:blue"><b>Respuesta</b></span> En este caso, la política es "latente", no definimos a mano el comportamiento de nuestra política; **es el entrenamiento el que indirectamente la definirá**. Ahora, dado que la política no está entrenada/aprendida, **necesitamos especificar su comportamiento**. Por ejemplo, si queremos una política que, dada la función de valor, realice acciones que siempre conduzcan a la mayor recompensa, crearemos una **política codiciosa**.

-----

<!-- <table>
    <tr>
        <td><img src="images_2/link_value_policy.png" width="600" data-align="center"></td>
    </tr>
</table> -->

Escribimos la función de valor para un estado $s$ bajo una política $\pi$ de la siguiente forma:

$$
V_{\pi}(s) = \mathbf{E}_{\pi}[G_{t}[S_{t} = s]]
$$

Para cada estado $s$, la función devuelve el retorno esperado si el agente empieza en ese estado y sigue la política los siguientes instantes de tiempo.

<table>
    <tr>
        <td><img src="images_2/state-value-function-2.png" width="600" data-align="center"></td>
    </tr>
</table>

El problema es que para calcular el valor de cada estado debemos **sumar todas las recompensas que el agente puede obtener si empieza en dicho estado**. Esto puede ser computacionalmente muy caro, especialmente si tenemos un gran número de estados posibles. 

<table>
    <tr>
        <td><img src="images_2/bellman2.png" width="600" data-align="center"></td>
    </tr>
</table>

Para remediarlo, existe la <span style="color:blue"><b>ecuación de Bellman</b></span>.

### 2.3.1 - Ecuación de Bellman

La ecuación de Bellman es una función recursiva inspirada en la [programación dinámica](https://es.wikipedia.org/wiki/Programaci%C3%B3n_din%C3%A1mica) que "acumula" la suma local de recompensas para cada estado de tal forma que podamos agilizar el proceso de cómputo. Consideramos cada estado como **la recompensa inmediata $R_{t+1}$ (es decir, la recompensa de pasar al siguiente estado) + el valor descontado del estado siguiente** ($\gamma * V(S_{t+1})$).

<table>
    <tr>
        <td><img src="images_2/bellman4.png" width="600" data-align="center"></td>
    </tr>
</table>

### 2.3.2 - *Monte Carlo* vs *Temporal Difference*

Existen dos estrategias con las que aprender nuestra función de valor:
* ***Monte Carlo***. Utiliza la experiencia obtenida de todo un episodio antes de aprender.
* ***Temporal Difference***. Utiliza la experiencia obtnida después de cada paso en el episodio para aprender.

#### *Monte Carlo*

Monte Carlo espera hasta terminar el episodio, calcula el retorno $G_{t}$ obtenido para cada paso $t$ realizado y utiliza esta información como referencia para actualizar el valor de cada estado $V(S_{t})$:

<table>
    <tr>
        <td><img src="images_2/monte-carlo-approach.jpg" width="600" data-align="center"></td>
    </tr>
</table>

Consideremos el juego del ratón y el gato presentado anteriormente como ejemplo:

* Ponemos un límite al episodio. Por ejemplo:
    * Si el gato se come al ratón
    * Si el ratón realiza > 10 pasos
* Siempre empezamos el episodio en el **mismo punto de inicio**.
* **El agente realizará acciones con respecto a su política especificada**. Por ejemplo, puede usar una política codiciosa *epsilon* (i.e., *Epsilon Greedy Strategy*), la cual alterna entre exploración y explotación de forma probabilística.
* En cada instante de tiempo del episodio, obtenemos la **recompensa para ese estado** y cual es el **siguiente estado al que ir**.
* Al final del episodio, **sumamos las recompensas que ha obtenido** el agente (para ver cuan bien lo ha hecho).
* **Actualizamos el valor de cada estado** $V(S_{t})$ según la fórmula.
* Comenzar un nuevo episodio con este nuevo conocimiento.

Consideremos el siguiente episodio de ejemplo para ver como realizamos estos pasos:

<table>
    <tr>
        <td><img src="images_2/MC-4p.jpg" width="600" data-align="center"></td>
    </tr>
</table>

Una vez terminado el episodio, tenemos una lista con los diferentes estados en los que ha estado el agente asi como las recompensas correspondientes. A partir de esto, podemos calcular el **retorno total** $G_{t}$ del episodio:

* $G_{t} = R_{t+1} + R_{t+2} + R_{t+3} + \dots$
* $G_{t} = 1 + 0 + 0 + 0 + 0 + 0 + 1 + 1 + 0 + 0 = 3$

Con esta información y si asumimos que la tasa de aprendizaje ($\alpha$) es $0.1$ podemos el valor del estado $S_{0}$:

* $V(S_{0})^{\text{new}} = V_{S_{0}} + \alpha * [G_{t} - V(S_{0})]$
* $V(S_{0})^{\text{new}} = 0 + 0.1 * [3 - 0] = 0.3$

Podemos repetir este mismo procedimiento para el resto de estados $S_{1}$, $S_{2}$, etc.

#### *Temporal Difference*

*Temporal Difference* (TD) simplemente espera una iteración (un instante de tiempo) $S_{t+1}$ para generar un **retorno parcial** y actualizar $V(S_{t})$ utilizando $R_{t+1}$ y ($\gamma * V(S_{t+1})$).

Dado que no hemos experimento todo el episodio, no tenemos $G_{t}$ (el retorno total). **El retorno parcial aproxima $G_{t}$ mediante la suma de la recomensa inmidiatamente posterior $R_{t+1}$ y el valor descontado del siguiente estado $\gamma * V(S_{t+1})$**:

<table>
    <tr>
        <td><img src="images_2/TD-1.jpg" width="600" data-align="center"></td>
    </tr>
</table>

Si tomamos el mismo ejemplo que en el caso de *Monte Carlo* (juego del ratón y el gato):
* Consideramos que es el comienzo del entrenamiento, asi que el valor de todo los estados es $0$.
* Nuestra tasa de aprendizaje $\alpha$ es $0.1$ y nuestra tasa de descuento $\gamma$ es 1 (no se descuenta, recordemos que $0$ implica máximo descuento).
* Nuestro ratón **decide explorar el entorno y toma una acción aleatoria**: e.g., va hacia la izquierda.
* Recibe una recompensa $R_{t+1}$ ya que come una pieza de queso.

Con esta información podemos actualizar el valor del estado $S_{0}$:
* $V(S_{0})^{\text{new}} = V(S_{0}) + \alpha * [R_{t+1} + \gamma * V(S_{1}) - V(S_{0})]$
* $V(S_{0})^{\text{new}} = 0 + 0.1 * [1 + 1 * 0 - 0] = 0.1$

El agente continuaria actualizando su función de valor mientras interactúa con el entorno. En este caso con la versión actualizada de $V(S_{0})$. Al igual que con Monte Carlo, una vez termine el episodio, podemos repetirlo para seguir aprendiendo.

## 2.4 - Q-Learning

## 2.5 - Deep Reinforcement Learning

## 2.6 - Construyendo un agente mediante una red DQNN

### 2.6.1 - Importación de librerías

### 2.6.2 - Construcción del agente

### 2.6.3 - Construcción del método `main()`